In [12]:
#importing reqiured libraries
import pandas as pd
import pypistats
import plotly.express as px

# Getting PyPI statistics to select library/framework

In [13]:
# List of popular Python WebSocket libraries/frameworks
websocket_libraries = [
    "websockets",
    "python-socketio",
    "autobahn",
    "tornado",
    "gevent-websocket",
    "channels",
    "flask-socketio",
    "twisted",
    "aiohttp"]

df = pd.DataFrame([])
for lib in websocket_libraries:
    # Download statistics 
    df_temp=  pypistats.overall(lib, total=True, format="pandas").dropna()
    df_temp = df_temp[df_temp['category']=='with_mirrors'].drop(['category','percent'],axis=1)
    df_temp['date'] = df_temp['date'].apply(pd.to_datetime)
    df_temp.rename(columns={'downloads': lib},inplace=True)
    df_temp.set_index('date',inplace=True)
    # Weekend downloads drops dramatically, so just drop them
    df_temp = df_temp[df_temp.index.day_of_week<5]
    df_temp = df_temp.sort_index()
    # Some MA just to smoorth graph 
    df_temp[lib] = df_temp[lib].rolling(3).mean()
    df = pd.concat([df, df_temp],axis=1)
df.dropna(inplace=True)

In [14]:
#plotting pypi datas
fig = px.line(df, x=df.index, y=df.columns)
fig.show()

# Starting Benchmarking

### AioHttp is the most downloaded and websocket is the second (with websocket-client), so I decided to benchmark with them

In [15]:
#Importing benchmark results csv
aiohttp_np1d      = pd.read_csv("aiohttp_np1d.csv").drop(["Unnamed: 0"],axis=1)
aiohttp_np2d      = pd.read_csv("aiohttp_np2d.csv").drop(["Unnamed: 0"],axis=1)
aiohttp_json1d    = pd.read_csv("aiohttp_json1d.csv").drop(["Unnamed: 0"],axis=1)
aiohttp_json2d    = pd.read_csv("aiohttp_json2d.csv").drop(["Unnamed: 0"],axis=1)
websocket_np1d    = pd.read_csv("websocket_np1d.csv").drop(["Unnamed: 0"],axis=1)
websocket_np2d    = pd.read_csv("websocket_np2d.csv").drop(["Unnamed: 0"],axis=1)
websocket_json1d  = pd.read_csv("websocket_json1d.csv").drop(["Unnamed: 0"],axis=1)
websocket_json2d  = pd.read_csv("websocket_json2d.csv").drop(["Unnamed: 0"],axis=1)

#Getting average of elapsed time of each iteration
aiohttp_np1d   = pd.DataFrame(aiohttp_np1d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'aiohttp'})
aiohttp_np2d   = pd.DataFrame(aiohttp_np2d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'aiohttp'})
aiohttp_json1d = pd.DataFrame(aiohttp_json1d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'aiohttp'})
aiohttp_json2d = pd.DataFrame(aiohttp_json2d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'aiohttp'})

websocket_np1d   = pd.DataFrame(websocket_np1d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'websocket'})
websocket_np2d   = pd.DataFrame(websocket_np2d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'websocket'})
websocket_json1d = pd.DataFrame(websocket_json1d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'websocket'})
websocket_json2d = pd.DataFrame(websocket_json2d.groupby('size')["elapsed_time"].mean()).rename(columns={"elapsed_time":'websocket'})


#concatting by data types
df_np1d   = pd.concat([websocket_np1d  ,aiohttp_np1d  ],axis=0)
df_np2d   = pd.concat([websocket_np2d  ,aiohttp_np2d  ],axis=0)
df_json1d = pd.concat([websocket_json1d,aiohttp_json1d],axis=0)
df_json2d = pd.concat([websocket_json2d,aiohttp_json2d],axis=0)


## Time to plot results

In [16]:
fig = px.line(data_frame=df_json1d, x=df_json1d.index, y=df_json1d.columns, title="Message of json with 1 dimensional array in it",labels={'value':"Rount-trip Time (s) "})
fig.show()

In [17]:
fig = px.line(data_frame=df_json2d, x=df_json2d.index, y=df_json2d.columns, title="Message of json with 2 dimensional array in it",labels={'value':"Rount-trip Time (s) "})
fig.show()

In [18]:
fig = px.line(data_frame=df_np1d, x=df_np1d.index, y=df_np1d.columns, title="Message of binary with 1 dimensional array in it",labels={'value':"Rount-trip Time (s) "})
fig.show()

In [19]:
fig = px.line(data_frame=df_np2d, x=df_np2d.index, y=df_np2d.columns, title="Message of binary with 2 dimensional array in it",labels={'value':"Rount-trip Time (s) "})
fig.show()

In [20]:
df_log_aio       = pd.read_csv("process_log_aiohttp.txt")
df_log_websocket = pd.read_csv("process_log_websocket.txt")

df_log_aio['Memory (MB)']   = df_log_aio['Memory (MB)'].apply(lambda x: int(x.split(":")[1].split("M")[0]))
df_log_aio['CPU Usage (%)'] = df_log_aio['CPU Usage (%)'].apply(lambda x: float(x.split(":")[1].split("%")[0]))
df_log_aio['CPU Time']      = df_log_aio['CPU Time'].apply(lambda x: int(x.split(":")[1])*60+int(x.split(":")[2]))
df_log_aio.set_index('CPU Time',inplace=True)

df_log_websocket['Memory (MB)']   = df_log_websocket['Memory (MB)'].apply(lambda x: int(x.split(":")[1].split("M")[0]))
df_log_websocket['CPU Usage (%)'] = df_log_websocket['CPU Usage (%)'].apply(lambda x: float(x.split(":")[1].split("%")[0]))
df_log_websocket['CPU Time']      = df_log_websocket['CPU Time'].apply(lambda x: int(x.split(":")[1])*60+int(x.split(":")[2]))
df_log_websocket.set_index('CPU Time',inplace=True)


In [21]:
fig = px.line(data_frame=df_log_websocket, x=df_log_websocket.index, y=df_log_websocket.columns, title="CPU and RAM usage of websocket_client.py (without pandas actions)")
fig.show()

In [22]:
fig = px.line(data_frame=df_log_aio, x=df_log_aio.index, y=df_log_aio.columns, title="CPU and RAM usage of aiohttp_client.py (without pandas actions)")
fig.show()